# **Outline:**
## The goal of this script is to create several Non-GaaP metrics that capture the health of Toast

The main non-GaaP metrics are
Net Retention Rate (NRR)
Cohort Analysis
ARR by Prouct Line

The majority of the script is used to create a few datasets, the high level bullet points are as follows
- Uses the monthly customer total saas table to get the total live saas and if the customers has any Guest modules
- Uses the monthly snapshots of directional payments and hourly volume to get gmv, gpv, and payments margin all annualized on a monthly basis
- For the payments metrics stated above, use the first 28 of the month to elimiunate day of week seasonality as simply as possible
- For EC data, use the ec guid on the account table - sometimes there are more than one guid on an account so logic needs to be implemented to strip these guids into list format
- Use the account segment table to get the join from a location to a parent
- Roll up the SaaS and Payments metrics on a monthly/location level to a monthly/parent level
- Combine this to EC data on a monthly/parent level
- This data can be used to see how a parent account changes over time, specifically how is changes year to year

In [2]:
import os
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from datetime import date
import statsmodels.api as sm
from matplotlib import dates
from IPython.core.display import display, HTML
import matplotlib.ticker as mtick
from matplotlib.ticker import MaxNLocator
import itertools
from dateutil.relativedelta import relativedelta
from matplotlib.ticker import FuncFormatter
import shutil
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

from datetime import datetime
import sys
warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import snowflake.connector
cnx = snowflake.connector.connect(
    user='nmittleman@toasttab.com',
    account='toast.us-east-1',
    authenticator='externalbrowser'
    )

%matplotlib inline

def QueryDWH(query, cnx, col=[]):
    cur = cnx.cursor()
    cur.execute(query)
    # If there is no col input, then get the columns from the return of the
    # snowflake query
    if not col:
        df = pd.DataFrame(cur.fetchall())
        if not df.empty:
            df.columns = [desc[0] for desc in cur.description]
    else:
        df = pd.DataFrame(cur.fetchall(), columns=col)

    return df
QueryDWH("""USE DATABASE TOAST""", cnx)
QueryDWH("""USE WAREHOUSE TOAST_WH""", cnx)



def get_quarter(date):
    return int((date.month - 1) / 3) + 1

def get_first_day_of_the_quarter(date):
    quarter = get_quarter(date)
    return datetime(date.year, int(3 * quarter), 1)

current_month = date(datetime.today().year, datetime.today().month, 1)
output_folder = os.path.join('./',f'Pitch Deck Outputs {str(datetime.now().date())}')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
# database = 'toast_clone_20210708'    
database = 'toast'    
# database = 'toast_clone_20220121'    
save_bool = True


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


## Creates the Net / Gross Retention Calculation Function

In [2]:
# parent_df - A monthly view of the parent accounts and their revenue breakdown
# save_bool - If True, save output
# output_folder - Where to save the outputs to
# file_name - Filename of the outputs
# retention_type - 'Net' for Net Retention or 'Gross' for Gross Retention
# save_raw - should the raw dataframe be included in the output
def retention(parent_df, save_bool, output_folder, file_name, retention_type = 'Net', save_raw = False):
    """
    Gets the Net Retention Rate (rr)
    This captures Upsells/Downsells, account expansion, and churn
    """
    
    # If we are looking monthly this is 1, if quarterly its 3
    num_months = 1
    
    parent_df['month'] = pd.to_datetime(parent_df['month']).dt.date
    parent_df['quarter'] = pd.to_datetime(parent_df['quarter']).dt.date

    """
    Handles partial month issues
    """
    
    max_month = current_month - relativedelta(months=12)
    max_month_minus_1 = max_month - relativedelta(months=1)
    final_month = parent_df[parent_df['month'] == max_month_minus_1]
    final_month['month'] = max_month
    rr = pd.concat([parent_df[parent_df['month'] < max_month], final_month])
    rr = pd.concat([parent_df])
    
    # If we are looking at Gross Revenue, cap the 12 month value at the current month value
    if retention_type == 'Gross':
        for col in ['pos_live_saas_arr_full_month', 'ec_saas_arr', 'payments_margin_arr_full_month', 'annual_gmv', 'annual_gpv',]:
            rr[col + ' 12 months'] = rr[[col, col + ' 12 months']].min(axis=1)

    """
    Get SaaS at the end of the period, but sum Payments info across the period - only difference is if it looks monthly vs quarterly
    """
    cols = ['pos_live_saas_arr_full_month', 'pos_live_saas_arr_full_month 12 months', 'ec_saas_arr', 'ec_saas_arr 12 months']
    rr_agg = rr.groupby('month')[cols].sum()

    cols = ['payments_margin_arr_full_month', 'payments_margin_arr_full_month 12 months', 'annual_gmv', 'annual_gmv 12 months', 'annual_gpv', 'annual_gpv 12 months']
    rr_agg_vol = rr.groupby(['month'])[cols].sum()


    """
    Because payments metrics are annualized, need to divide by num_months to handle the quarters
    """
    rr_agg_vol['Payments ARR'] = rr_agg_vol['payments_margin_arr_full_month'] / (num_months)
    rr_agg_vol['Payments ARR 12 Months'] = rr_agg_vol['payments_margin_arr_full_month 12 months']  / (num_months)
    rr_agg_vol['annual_gmv'] = rr_agg_vol['annual_gmv']  / ( num_months)
    rr_agg_vol['annual_gmv 12 months'] = rr_agg_vol['annual_gmv 12 months']  / (num_months)
    rr_agg_vol['annual_gpv'] = rr_agg_vol['annual_gpv'] / (num_months)
    rr_agg_vol['annual_gpv 12 months'] = rr_agg_vol['annual_gpv 12 months'] / (num_months)

    """
    Combine the SaaS and Payments data on a period basis (only difference is if monthly vs quarterly)
    """
    rr_agg = rr_agg.join(rr_agg_vol)
    # Create user friendly names
    rr_agg.rename(columns = {'pos_live_saas_arr_full_month': 'Live POS SaaS', 'pos_live_saas_arr_full_month 12 months': 'Live POS SaaS 12 Months',
                              'ec_saas_arr': 'Live EC SaaS', 'ec_saas_arr 12 months': 'Live EC SaaS 12 Months',
                             'annual_gmv': 'GMV', 'annual_gmv 12 months': 'GMV 12 Months',
                             'annual_gpv': 'GPV', 'annual_gpv 12 months': 'GPV 12 Months'}, inplace=True)

    rr_agg['Live POS SaaS']= rr_agg['Live POS SaaS'] 
    rr_agg['Live EC SaaS']= rr_agg['Live EC SaaS']
    rr_agg['Live POS SaaS 12 Months']= rr_agg['Live POS SaaS 12 Months']
    rr_agg['Live EC SaaS 12 Months']= rr_agg['Live EC SaaS 12 Months']

    rr_agg['Starting Live SaaS'] = (rr_agg['Live POS SaaS'] + rr_agg['Live EC SaaS'])
    rr_agg['Ending Live SaaS'] = (rr_agg['Live POS SaaS 12 Months'] + rr_agg['Live EC SaaS 12 Months'])

    # Calc Retention Rates as value 12 months later over value in month
    rr_agg['Net SaaS Retention Rate'] =  rr_agg['Ending Live SaaS'] / rr_agg['Starting Live SaaS']
    rr_agg['Net Payments ARR Retention Rate'] =  rr_agg['Payments ARR 12 Months'] / rr_agg['Payments ARR']
    rr_agg['Net GMV Retention Rate'] =  rr_agg['GMV 12 Months'] / rr_agg['GMV']
    rr_agg['Net GPV Retention Rate'] =  rr_agg['GPV 12 Months'] / rr_agg['GPV']

    # This just removes incomplete rows
    # rr_agg = rr_agg[rr_agg['Net SaaS Retention Rate'] > 0]

    # Reorganize columns
    rr_agg = rr_agg[['Live POS SaaS', 'Live POS SaaS 12 Months', 'Live EC SaaS', 'Live EC SaaS 12 Months'
                       , 'Starting Live SaaS', 'Ending Live SaaS', 'Net SaaS Retention Rate'
                       , 'Payments ARR', 'Payments ARR 12 Months', 'Net Payments ARR Retention Rate'
                       , 'GMV', 'GMV 12 Months', 'Net GMV Retention Rate'
                       , 'GPV', 'GPV 12 Months', 'Net GPV Retention Rate']]

    if save_bool:
        writer = pd.ExcelWriter(os.path.join(output_folder, file_name), engine='xlsxwriter')
        rr_agg.to_excel(writer, sheet_name='Aggregate', index=True)
        if save_raw:
            rr.to_excel(writer, sheet_name='Raw Data', index=True)
        writer.save()    
    return rr_agg, rr

## Get EC - SFDC connections
Get all the EC guids tied to SFDC accounts and unpack them

In [3]:
query = f"""
        SELECT account_id, salesforce_accountid, ec_customer_guid
        FROM {database}.analytics_core.account a
        WHERE ec_customer_guid IS NOT NULL
        """
sfdc_ec = QueryDWH(query, cnx, ['snowflake_id','salesforce_accountid', 'ec_guid'])

### Pull account level information from the account / customer table in snowflake - based off of salesforce ids

In [4]:
"""
For all accounts, get the account info such as id, market segments, and parent id 
"""
query = f"""
        SELECT a.account_id
             , a.salesforce_accountid
             , a.customer_name
             , c.churn_reason
             , c.churn_date
             , DATE_TRUNC('Month' ,c.churn_date)
             , COALESCE(b.highest_parent_id, MD5_HEX(a.salesforce_parent_accountid)) AS parent_id
             , COALESCE(b.current_market_segment, a.parent_market_segment) AS market_segment
             , c.pos_first_order_date
             , MIN(pos_first_order_date) OVER (PARTITION BY parent_id) AS first_go_live_parent
             , COALESCE(c.toastorders_state, a.state) AS state
             , COALESCE(c.restaurant_type, a.restaurant_type) AS restaurant_type
        FROM {database}.analytics_core.account a
        LEFT JOIN {database}.analytics_core.account_segment b
        ON a.account_id = b.account_id
        LEFT JOIN analytics_core.customer c
        ON a.account_id = c.customer_id  
        """

acct_info = QueryDWH(query, cnx, ['customer_id', 'sf_accountid', 'customer_name', 'churn_reason', 'churn_date', 'churn_month' ,'highest_parent_id', 'market_segment', 'fod','parent_first_go_live', 'state', 'restaurant_type'])
acct_info['fod'] = pd.to_datetime(acct_info['fod']).dt.date
acct_info['parent_first_go_live'] = pd.to_datetime(acct_info['parent_first_go_live']).dt.date
acct_info['churn_date'] = pd.to_datetime(acct_info['churn_date']).dt.date
acct_info['churn_month'] = pd.to_datetime(acct_info['churn_month']).dt.date
acct_info['market_segment'].fillna('SMB', inplace=True)
acct_info['state'].fillna('Unknown', inplace=True)

In [5]:
"""
Sometimes ec guids in salesforce are actually lists, unfortunately, there is no consistency yet how they are delimited so 
this function breaks them apart and creates new rows in a dataframe for every ec_guid
"""
def split_guids(sfdc_ec):
    sfdc_ec['is_list'] = np.where(sfdc_ec['ec_guid'].str.contains(','), True, False)
    sfdc_ec['is_list'] = np.where(sfdc_ec['ec_guid'].str.contains(';'), True, sfdc_ec['is_list'])
    sfdc_ec['is_list'] = np.where(sfdc_ec['ec_guid'].str.contains(':'), True, sfdc_ec['is_list'])
    sfdc_ec['is_list'] = np.where(sfdc_ec['ec_guid'].str.contains('/'), True, sfdc_ec['is_list'])
    sfdc_ec['is_list'] = np.where(sfdc_ec['ec_guid'].str.contains(' '), True, sfdc_ec['is_list'])


    sfdc_ec['delim'] = np.where(sfdc_ec['ec_guid'].str.contains(' '), ' ', None)
    sfdc_ec['delim'] = np.where(sfdc_ec['ec_guid'].str.contains(';'), ';', sfdc_ec['delim'])
    sfdc_ec['delim'] = np.where(sfdc_ec['ec_guid'].str.contains(':'), ':', sfdc_ec['delim'])
    sfdc_ec['delim'] = np.where(sfdc_ec['ec_guid'].str.contains(','), ',', sfdc_ec['delim'])
    sfdc_ec['delim'] = np.where(sfdc_ec['ec_guid'].str.contains('/'), '/', sfdc_ec['delim'])

    not_list = sfdc_ec[sfdc_ec['is_list'] == False]
    not_list['split'] = False
    is_list = sfdc_ec[sfdc_ec['is_list']]
    is_list['list_len'] = is_list.apply(lambda v: len(v['ec_guid'].split(v['delim'])), axis=1)
    multi_guid = pd.DataFrame()
    for i in range(0, is_list['list_len'].max()):
        tmp = is_list.copy()
        tmp['ec_guid'] = tmp.apply(lambda v: v['ec_guid'].split(v['delim'])[i] if i < v['list_len']  else None , axis=1)
        multi_guid = pd.concat([multi_guid, tmp])
    multi_guid['split'] = True
    cols = ['snowflake_id', 'salesforce_accountid', 'ec_guid', 'split']
    return pd.concat([not_list[cols], multi_guid[cols]])

# Get all of the instances of accounts with multiple EC guids tied to them, and separate into multiple rows
tmp = split_guids(sfdc_ec)
tmp = split_guids(tmp)
ec_mapping = tmp[tmp['ec_guid'].fillna('').str.contains('-')].drop('split', axis=1)
ec_mapping = ec_mapping[['ec_guid', 'snowflake_id', 'salesforce_accountid']]
ec_mapping = ec_mapping.sort_values(['ec_guid']).drop_duplicates(['ec_guid', 'snowflake_id', 'salesforce_accountid'], keep='first')



In [6]:
# ec_mapping contains the ec guid, the corresponding snowflake account, and the salesforce account (could be a parent or a location)
ec_mapping.head()

,ec_guid,snowflake_id,salesforce_accountid
3169,a7c4a2cc-8be6-11eb-b208-129eca3c4799,a343dfcbaba647aea4c31dd287a9a411,001C000001XNsV9IAL
3678,060722ef-b99a-11eb-b208-129eca3c4799,d86d7be53d1c41128329c4ace0cc9e28,0013c00001ttwYTAAY
11667,074954a9-ac19-11eb-b208-129eca3c4799,b321e469e5136c3f868af91347b3b944,0013c00001rBqnqAAC
9767,24ca9d7a-aaf4-11ec-978a-02c7d7953193,14fd79b0674aa521687475d6f5eafe57,0013c00001tnYwxAAE
4946,25cc816c-dc48-11ec-af2b-128b390ae63d,b4f778941d6293688eb7d86e17780da2,001C000001eHNTlIAO


## Get All EC Customers and EC ARR

In [7]:
"""
get the customer id's for all ec customers
"""
query = f"""
        SELECT ec_customer_id
             , ec_customer_guid
             , ec_customer_name
             , first_check_date
        FROM {database}.ec_core.ec_customer 
        """
ec_accts = QueryDWH(query, cnx, ['ec_id','ec_guid', 'ec_name','first_check'])

In [8]:
# high level ec account info. the id, the name and when the account went live
ec_accts.head()

,ec_id,ec_guid,ec_name,first_check
0,estratex_db5_l2_12095,8cbfcfa3-09e0-11ed-8360-128b390ae63d,Refinery Jax Beach,2022-08-05
1,estratex_db5_l2_12094,971bc3b8-09dd-11ed-8360-128b390ae63d,Pop Stop,None
2,estratex_db5_l2_12097,9cd4becd-09e1-11ed-8360-128b390ae63d,Farmer and the Dail,None
3,estratex_db5_l2_12101,a70f94c0-09e8-11ed-8360-128b390ae63d,Pita Pita Mediterranean Grill,None
4,estratex_db5_l2_12092,840847dd-09db-11ed-8360-128b390ae63d,Delicious Food Corner,None


In [9]:
"""
On a monthly basis, get the recurring saas on a ec customer level
"""
query = f"""
        SELECT DATE_TRUNC('Month', check_date), ec_customer_guid, a.ec_customer_id, SUM(total) * 12 AS ec_saas
        FROM {database}.ec_revenue.ec_pay_period_item_invoiced a
        LEFT JOIN {database}.ec_core.ec_customer b
        ON a.ec_customer_id = b.ec_customer_id
        WHERE recurring_flag = 'Recurring'
        GROUP BY 1,2,3;
        """
ec_monthly = QueryDWH(query, cnx, ['month','ec_guid', 'ec_id', 'ec_saas_arr'])
ec_monthly['month'] = pd.to_datetime(ec_monthly['month']).dt.date


ec_parent = ec_mapping.rename(columns = {'snowflake_id': 'customer_id'}).merge(acct_info[['customer_id', 'highest_parent_id']], how='left')
ec_monthly_parent_flat = ec_monthly.merge(ec_accts, how='left').merge(ec_parent, how='left')

ec_monthly_parent_flat = ec_monthly_parent_flat[ec_monthly_parent_flat['month'] >= date(2019,7,1)]
ec_monthly_parent = ec_monthly_parent_flat.groupby(['month', 'highest_parent_id'])[['ec_saas_arr']].sum().reset_index()
# only want ec data after the acquisition (July 2019)



## Get Monthly POS SaaS 
Get the amount of POS SaaS on a location basis

In [10]:
"""
Using the monthly customer total saas table, get monthly status and saas on a customer level

"""
query = f"""
        SELECT DATE_TRUNC('Month', a.dt) AS month
             , a.salesforce_accountid
             , MD5_HEX(a.salesforce_accountid) AS customer_id
             , a.pos_status
             , a.pos_first_order_date
             , a.nbo_close_date
             , IFF(DATE_TRUNC('Month', a.pos_first_order_date) = DATE_TRUNC('Month', a.dt), True, False) AS pos_go_live_this_month
             , IFF(DATE_TRUNC('Month', a.nbo_close_date) = DATE_TRUNC('Month', a.dt), True, False) AS pos_booked_this_month
             , a.pos_live_saas_amount * 12 AS pos_live_saas_arr
             , a.pos_backlog_saas_amount * 12 AS pos_backlog_saas_arr             
             , a.has_live_guest_saas
        FROM {database}.payments.daily_customer_details a
        WHERE a.dt = LAST_DAY(a.dt);
        """

pos_saas_month = QueryDWH(query, cnx, ['month', 'salesforce_accountid', 'customer_id', 'status', 'fod', 'close_date','go_live','booked', 'pos_live_saas_arr', 'pos_backlog_saas_arr', 'has_guest_saas'])
pos_saas_month['pos_live_saas_arr'] = pos_saas_month['pos_live_saas_arr'].astype(float)
pos_saas_month['has_guest_saas'] = pos_saas_month['has_guest_saas'].astype(float)

pos_saas_month['month'] = pd.to_datetime(pos_saas_month['month']).dt.date

pos_saas_month['fod'] = pd.to_datetime(pos_saas_month['fod']).dt.date
pos_saas_month['close_date'] = pd.to_datetime(pos_saas_month['close_date']).dt.date
pos_saas_month['quarter'] = pd.to_datetime(pos_saas_month['month'].apply(lambda v: get_first_day_of_the_quarter(v))).dt.date
pos_saas_month['12 months'] = pd.to_datetime(pos_saas_month['month']).dt.date + relativedelta(months=+12)

In [11]:
# Create a monthly/location dataset. has status info on the accounts, the saas, and if it has guest modules
pos_saas_month.head()

,month,salesforce_accountid,customer_id,status,fod,close_date,go_live,booked,pos_live_saas_arr,pos_backlog_saas_arr,has_guest_saas,quarter,12 months
0,2020-06-01,001C000001Iqq4vIAB,9dc98740c1cc6bd13641834924f26a58,Live,2014-07-28,2014-07-28,False,False,1980.0,0.0,1.0,2020-06-01,2021-06-01
1,2019-05-01,0013c00001o6shGAAQ,37ba70ae489ec16061e008f3b6c3231a,None,NaT,2019-07-24,False,False,0.0,0.0,0.0,2019-06-01,2020-05-01
2,2014-09-01,001C000001IqpzIIAR,c1a032119bd9a8d3a988b5224f5fdd04,None,2019-05-06,2019-01-31,False,False,0.0,0.0,0.0,2014-09-01,2015-09-01
3,2018-01-01,001C000001eIs49IAC,ef286922c515ccc7828c8e6811a96a6a,Live,2017-12-07,2017-11-28,False,False,2232.0,0.0,0.0,2018-03-01,2019-01-01
4,2021-09-01,001C000001imJWeIAM,23d32f27370333cb9fb399bdbbc632b7,Live,2018-09-16,2018-06-20,False,False,4590.0,0.0,1.0,2021-09-01,2022-09-01


In [12]:
# Get a mapping of snowflake accounts to the highest level parent. also contains some account info such as market segment, state and account go-live
acct_info.head()

,customer_id,sf_accountid,customer_name,churn_reason,churn_date,churn_month,highest_parent_id,market_segment,fod,parent_first_go_live,state,restaurant_type
0,215555d29d84977494d144d536c5aa3f,0013c00001uhZQ6AAM,Shorty's Pins x Pints Parent,None,NaT,NaT,fd98957f3f6a57c8a9150f5ee514c3f0,SMB,NaT,2022-02-15,Unknown,Unknown
1,c6ce01f41cfe65609040cd2a43f2c117,0014W00002hCYRLQA4,AOC Hospitality Parent,None,NaT,NaT,c6ce01f41cfe65609040cd2a43f2c117,SMB,NaT,2022-01-06,Unknown,Unknown
2,52e995a61b3b36f140d4e4c5219fe09f,0013c00001uPSRAAA4,the cloud Parent,None,NaT,NaT,52e995a61b3b36f140d4e4c5219fe09f,SMB,NaT,2021-11-24,NC,Fast Casual
3,2a54ab641c321b210bab75bde18d8a30,0013c00001sGywYAAS,"Family Style, Inc",None,NaT,NaT,2a54ab641c321b210bab75bde18d8a30,SMB,NaT,NaT,CA,FSR - Casual
4,701ecf071ef60e308d2adfc646ce953d,0014W00002Rox9CQAR,Caroline’s By The Bay Parent,None,NaT,NaT,701ecf071ef60e308d2adfc646ce953d,SMB,NaT,2021-06-30,NJ,Fast Casual


## Get Payments Data
Payments Margin
GMV
GPV

All metrics use the first 28 days of the month to remove day of week seasonality

In [13]:
"""
Get the annual gmv on a monthly basis by location
"""
query = f"""
        SELECT DATE_TRUNC('Month', a.dt) AS month
             , MD5_HEX(a.salesforce_accountid) AS customer_id

             , SUM(a.gmv) * 12 AS annual_gmv
             , SUM(a.gmv) AS monthly_gmv
        FROM {database}.payments.daily_customer_details a
        GROUP BY 1,2;
        """

gmv_month = QueryDWH(query, cnx, ['month', 'customer_id', 'annual_gmv', 'monthly_gmv'])
gmv_month['annual_gmv'] = gmv_month['annual_gmv'].astype(float)
gmv_month['monthly_gmv'] = gmv_month['monthly_gmv'].astype(float)

gmv_month['month'] = pd.to_datetime(gmv_month['month']).dt.date


In [14]:
# Get annual gmv on a monthly/location grain
gmv_month.head()

,month,customer_id,annual_gmv,monthly_gmv
0,2021-03-01,2dd51ff2730d98513d5d438bc7703e20,0.00,0.00
1,2020-05-01,f4ddcf2bbff9b0c88b813a7ad35c940d,1526539.80,127211.65
2,2021-11-01,cd36edbc8a587e28f0a9df59238a63cd,758226.72,63185.56
3,2021-09-01,ac61c0fb5392634df890d9e2f12e13c2,40360.32,3363.36
4,2020-04-01,d4a65a06fe676dc5a14532291854c949,0.00,0.00


In [15]:
query = f"""
        SELECT DATE_TRUNC('Month', a.dt) AS month
             , MD5_HEX(a.salesforce_accountid) AS customer_id
             , SUM(a.gpv_dpf) * 12 AS annual_gpv
             , SUM(a.gpv_dpf) AS monthly_gpv

             , SUM(a.margin) * 12 AS margin_arr
             , SUM(a.margin) AS monthly_margin
             
             , SUM(TDS_TRANSACTIONS) AS tds_tx
             , SUM(DD_TRANSACTIONS) AS dd_tx
             , SUM(GH_TRANSACTIONS) AS gh_tx
             , SUM(UE_TRANSACTIONS) AS ue_tx
             
        FROM {database}.payments.daily_customer_details a
        WHERE pos_status = 'Live'
        GROUP BY 1,2;
        """

gpv_month = QueryDWH(query, cnx, ['month', 'customer_id', 'annual_gpv', 'monthly_gpv', 'payments_margin_arr', 'monthly_payments_margin', 
                                  'tds_tx', 'dd_tx', 'gh_tx', 'ue_tx'])
gpv_month['annual_gpv'] = gpv_month['annual_gpv'].astype(float)
gpv_month['monthly_gpv'] = gpv_month['monthly_gpv'].astype(float)
gpv_month['annual_gpv'] = gpv_month['annual_gpv'].astype(float)
gpv_month['payments_margin_arr'] = gpv_month['payments_margin_arr'].astype(float)
gpv_month['monthly_payments_margin'] = gpv_month['monthly_payments_margin'].astype(float)

gpv_month['tds_tx'] = gpv_month['tds_tx'].fillna(0).astype(float)
gpv_month['dd_tx'] = gpv_month['dd_tx'].fillna(0).astype(float)
gpv_month['gh_tx'] = gpv_month['gh_tx'].fillna(0).astype(float)
gpv_month['ue_tx'] = gpv_month['ue_tx'].fillna(0).astype(float)

gpv_month['month'] = pd.to_datetime(gpv_month['month']).dt.date

## Adjust For Payment Rate Matches
gpv_month.sort_values(['customer_id','month'], inplace=True)
gpv_month['month_num'] = gpv_month.groupby(['customer_id']).cumcount()+1
gpv_month['margin_bps'] = gpv_month['payments_margin_arr'] / gpv_month['annual_gpv'] * 10000


In [18]:
# Calculate Rev Share ARR/MRR

gpv_month['tds_cx'] = np.where(gpv_month['tds_tx'] > 0, 1, 0)
monthly_rev_share = gpv_month.groupby('month').agg({'tds_cx': 'sum', 'gh_tx': 'sum'}).reset_index()

# TDS Mult
monthly_rev_share['tds_mult'] = np.where(monthly_rev_share['month'] < date(2021,3,1), 0.65, 0)
monthly_rev_share['tds_mult'] = np.where((monthly_rev_share['month'] >= date(2021,3,1)) & \
                                         (monthly_rev_share['tds_cx'] <= 2000),
                                         0.78, monthly_rev_share['tds_mult'])
monthly_rev_share['tds_mult'] = np.where((monthly_rev_share['month'] >= date(2021,3,1)) & \
                                         (monthly_rev_share['tds_cx'] > 2000) & \
                                         (monthly_rev_share['tds_cx'] <= 5000),
                                         0.81, monthly_rev_share['tds_mult'])
monthly_rev_share['tds_mult'] = np.where((monthly_rev_share['month'] >= date(2021,3,1)) & \
                                         (monthly_rev_share['tds_cx'] > 5000) & \
                                         (monthly_rev_share['tds_cx'] <= 10000),
                                         0.84, monthly_rev_share['tds_mult'])
monthly_rev_share['tds_mult'] = np.where((monthly_rev_share['month'] >= date(2021,3,1)) & \
                                         (monthly_rev_share['tds_cx'] > 10000) & \
                                         (monthly_rev_share['tds_cx'] <= 20000),
                                         0.87, monthly_rev_share['tds_mult'])
monthly_rev_share['tds_mult'] = np.where((monthly_rev_share['month'] >= date(2021,3,1)) & \
                                         (monthly_rev_share['tds_cx'] > 20000),
                                         0.90, monthly_rev_share['tds_mult'])

# DoorDash
monthly_rev_share['dd_mult'] = np.where(monthly_rev_share['month'] >= date(2021,10,1), 0.17, 0)

# UberEats
monthly_rev_share['ue_mult'] = np.where(monthly_rev_share['month'] >= date(2021,10,1), 0.08, 0)

# GrubHub
monthly_rev_share['gh_mult'] = np.where(monthly_rev_share['month'] < date(2021,10,1), 0, 0)
monthly_rev_share['gh_mult'] = np.where((monthly_rev_share['month'] >= date(2021,10,1)) & \
                                         (monthly_rev_share['gh_tx'] <= 200000),
                                         0.15, monthly_rev_share['gh_mult'])
monthly_rev_share['gh_mult'] = np.where((monthly_rev_share['month'] >= date(2021,10,1)) & \
                                         (monthly_rev_share['gh_tx'] > 200000) & \
                                         (monthly_rev_share['gh_tx'] <= 400000),
                                         0.10, monthly_rev_share['gh_mult'])
monthly_rev_share['gh_mult'] = np.where((monthly_rev_share['month'] >= date(2021,10,1)) & \
                                         (monthly_rev_share['gh_tx'] > 400000),
                                         0.05, monthly_rev_share['gh_mult'])


gpv_month.drop(['tds_mult', 'dd_mult', 'ue_mult', 'gh_mult'], axis=1, inplace=True, errors='ignore')
gpv_month = gpv_month.merge(monthly_rev_share[['month','tds_mult', 'dd_mult', 'ue_mult', 'gh_mult']], how='left')
gpv_month['monthly_tds_rev_share'] = gpv_month['tds_tx'] * gpv_month['tds_mult']
gpv_month['monthly_dd_rev_share'] = gpv_month['dd_tx'] * gpv_month['dd_mult']
gpv_month['monthly_gh_rev_share'] = gpv_month['gh_tx'] * gpv_month['gh_mult']
gpv_month['monthly_ue_rev_share'] = gpv_month['ue_tx'] * gpv_month['ue_mult']

gpv_month['tds_rev_share_arr'] = gpv_month['monthly_tds_rev_share'].fillna(0) * 12
gpv_month['dd_rev_share_arr'] = gpv_month['monthly_dd_rev_share'].fillna(0) * 12
gpv_month['gh_rev_share_arr'] = gpv_month['monthly_gh_rev_share'].fillna(0) * 12
gpv_month['ue_rev_share_arr'] = gpv_month['monthly_ue_rev_share'].fillna(0) * 12

gpv_month['rev_share_arr'] = gpv_month[['tds_rev_share_arr', 'dd_rev_share_arr', 'gh_rev_share_arr', 'ue_rev_share_arr']].sum(axis=1)

## Get Monthly POS Location and Parent Level Info

In [19]:
# Join POS SaaS and Payments Data on the monthly location level
location_pos_monthly = pos_saas_month.merge(acct_info[['customer_id', 'highest_parent_id', 'churn_reason', 'churn_date', 'churn_month']], how='left').merge(gmv_month, how='left').merge(gpv_month, how='left')

# Fill in nulls and create a few columns from statuses to make counting easier
location_pos_monthly['annual_gmv'].fillna(0, inplace=True)
location_pos_monthly['annual_gpv'].fillna(0, inplace=True)
location_pos_monthly['payments_margin_arr'].fillna(0, inplace=True)
location_pos_monthly['monthly_gmv'].fillna(0, inplace=True)
location_pos_monthly['monthly_gpv'].fillna(0, inplace=True)
location_pos_monthly['monthly_payments_margin'].fillna(0, inplace=True)


location_pos_monthly['fod_month'] = pd.to_datetime(location_pos_monthly['fod'].fillna(date(2000,1,1))).apply(lambda v: date(int(v.year), int(v.month), 1))
location_pos_monthly['status'] = np.where(location_pos_monthly['fod_month'] == location_pos_monthly['month'], 'Go-Live', location_pos_monthly['status'])

location_pos_monthly['live_count'] = np.where(location_pos_monthly['status'] == 'Live', 1, 0)
location_pos_monthly['backlog_count'] = np.where(location_pos_monthly['status'] == 'Backlog', 1, 0)
location_pos_monthly['go_live_count'] = np.where(location_pos_monthly['go_live'], 1, 0)
location_pos_monthly['booked_count'] = np.where(location_pos_monthly['booked'], 1, 0)

location_pos_monthly['pos_live_saas_arr'] = location_pos_monthly['pos_live_saas_arr'] + location_pos_monthly['rev_share_arr'].fillna(0)
# Only look at full months of data
location_pos_monthly['pos_live_saas_arr_full_month'] = np.where((location_pos_monthly['status'].isin(['Live', 'Go-Live'])) & (location_pos_monthly['fod'] < location_pos_monthly['month']), location_pos_monthly['pos_live_saas_arr'], 0)
location_pos_monthly['payments_margin_arr_full_month'] = np.where((location_pos_monthly['status'].isin(['Live', 'Go-Live'])) & (location_pos_monthly['fod'] < location_pos_monthly['month']), location_pos_monthly['payments_margin_arr'], 0)

# See if the location had churned 12 months down the road
location_pos_monthly = location_pos_monthly.\
    merge(location_pos_monthly[['customer_id', 'month', 'status', 'live_count']].\
          rename(columns = {'month': '12 months','status': 'status 12 months', 'live_count': 'is live 12 months'}),
         how='left')

location_pos_monthly['month'] = pd.to_datetime(location_pos_monthly['month']).dt.date
location_pos_monthly['churn_month'] = pd.to_datetime(location_pos_monthly['churn_month']).dt.date
location_pos_monthly['status'] = np.where(  location_pos_monthly['month'] >= location_pos_monthly['churn_month'] , 'Churn',  location_pos_monthly['status'])


In [20]:
# Create two location monthly dataframes. one where even locations churned in 12 months are included, and the other they are excluded

location_pos_monthly_keeping_churn = location_pos_monthly[(location_pos_monthly['status'].isin(['Backlog', 'Live', 'Go-Live']))]
location_pos_monthly_removing_churn =  location_pos_monthly[(location_pos_monthly['status'].isin(['Backlog', 'Live', 'Go-Live'])) & (location_pos_monthly['status 12 months'] != 'Churn') & (location_pos_monthly['churn_reason'] != 'Competitor')]


## Parent Monthly Dataset
This dataset has monthly SaaS, Payments and EC info on a parent basis

In [21]:
def create_monthly_parent_df(location_pos_monthly, ec_monthly_parent, acct_info):
    
    # Monthly POS Data on the Parent Account Level
    parent_pos_monthly = location_pos_monthly.groupby(['highest_parent_id', 'month', '12 months'])[['pos_live_saas_arr_full_month', 'payments_margin_arr_full_month',  'annual_gmv', 'annual_gpv', 'live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']].sum().reset_index()
    parent_info = acct_info[acct_info['highest_parent_id'] == acct_info['customer_id']][['customer_id', 'customer_name', 'highest_parent_id', 'market_segment', 'parent_first_go_live']]
    parent_pos_monthly = parent_pos_monthly.merge(parent_info, how='inner')
    
    parent_monthly_all = parent_pos_monthly.merge(ec_monthly_parent, on=['highest_parent_id', 'month'], how='left').drop('customer_id', axis=1)
    parent_monthly_all.rename(columns = {'highest_parent_id': 'snowflake_id', 'account_id': 'salesforce_accountid'},inplace=True)
    parent_monthly_all['ec_saas_arr'].fillna(0, inplace=True)

    """
    Combine the parent monthly dataset with a copy of itself, offset 12 months
    This allows us to to see how the parent account changes over that time period
    """
    parent_monthly_all = parent_monthly_all[['snowflake_id', 'customer_name', 'market_segment', 'parent_first_go_live','month','12 months', 'pos_live_saas_arr_full_month', 'ec_saas_arr', 'payments_margin_arr_full_month','annual_gmv', 'annual_gpv','live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']]
    parent_monthly_12 = parent_monthly_all[['snowflake_id', 'month', 'pos_live_saas_arr_full_month', 'ec_saas_arr', 'payments_margin_arr_full_month', 'annual_gmv', 'annual_gpv','live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']].rename(columns={'month': '12 months'})
    col_12_month_list = []
    for col in ['pos_live_saas_arr_full_month', 'ec_saas_arr', 'payments_margin_arr_full_month','annual_gmv', 'annual_gpv','live_count', 'backlog_count', 'go_live_count', 'booked_count','has_guest_saas']:
        col_12_month_list.append(f'{col} 12 months')
        parent_monthly_12.rename(columns = {col: f'{col} 12 months'}, inplace=True)
    parent_monthly_all = parent_monthly_all.merge(parent_monthly_12, how='left')

    for col in col_12_month_list:
        parent_monthly_all[col].fillna(0, inplace=True)

    """
    Don't include the first month of a location - avoid partial motnhs
    """
    parent_monthly_all = parent_monthly_all[parent_monthly_all['parent_first_go_live'].notnull()]
    parent_monthly = parent_monthly_all[parent_monthly_all['month'] > parent_monthly_all['parent_first_go_live']]
    parent_monthly['first_full_month'] = parent_monthly['parent_first_go_live'] + relativedelta(months=1)
    parent_monthly['first_full_month'] = parent_monthly['first_full_month'].apply(lambda v: date(v.year, v.month, 1))
    parent_monthly['cohort_year'] = pd.to_datetime(parent_monthly['first_full_month']).dt.year
    parent_monthly['quarter'] = parent_monthly['month'].apply(lambda v: get_first_day_of_the_quarter(v))
    return parent_monthly

In [22]:
parent_monthly_keeping_churn = create_monthly_parent_df(location_pos_monthly_keeping_churn, ec_monthly_parent, acct_info)
parent_monthly_removing_churn = create_monthly_parent_df(location_pos_monthly_removing_churn, ec_monthly_parent, acct_info)

In [23]:
"""
Similar logic to the parent monthly table created above. 
Main difference is that EC revenue is prorated across all locations under that parent
We can't tie an EC location to a POS location, so the assumption is that all locations under a parent account have access to EC if there is EC on the parent
We know this is not 100% correct, but should get us close on the ARPU side for SMB
"""
location_monthly = location_pos_monthly_keeping_churn[['customer_id', 'month','12 months', 'pos_live_saas_arr_full_month', 'rev_share_arr' , 'payments_margin_arr_full_month',  'annual_gmv', 'annual_gpv', 'live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']].rename(columns = {'customer_id': 'snowflake_id'})
location_monthly_12 = location_pos_monthly_keeping_churn[['customer_id', 'month', 'pos_live_saas_arr_full_month', 'rev_share_arr' , 'payments_margin_arr_full_month',  'annual_gmv', 'annual_gpv', 'live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']].rename(columns={'month': '12 months', 'customer_id': 'snowflake_id'})
col_12_month_list = []
for col in [ 'pos_live_saas_arr_full_month', 'rev_share_arr' ,'payments_margin_arr_full_month',  'annual_gmv', 'annual_gpv', 'live_count', 'backlog_count', 'go_live_count', 'booked_count', 'has_guest_saas']:
    col_12_month_list.append(f'{col} 12 months')
    location_monthly_12.rename(columns = {col: f'{col} 12 months'}, inplace=True)

location_monthly = location_monthly.merge(location_monthly_12, how='left')

for col in col_12_month_list:
    location_monthly[col].fillna(0, inplace=True)

location_monthly = location_monthly.merge(acct_info[['customer_id', 'customer_name', 'highest_parent_id', 'market_segment', 'parent_first_go_live']].rename(columns = {'customer_id': 'snowflake_id'}), on='snowflake_id', how='left')
location_monthly = location_monthly[location_monthly['live_count'] > 0]

location_monthly = location_monthly.merge(parent_monthly_keeping_churn[['snowflake_id', 'month', 'ec_saas_arr', 'ec_saas_arr 12 months']].rename(columns = {'snowflake_id': 'highest_parent_id'}), on=['highest_parent_id', 'month'], how='left')

"""
Get the number of live locations on the parent, use that to prorate EC saas across the child locations
"""
location_monthly['num live locations on parent'] = location_monthly.groupby(['highest_parent_id', 'month'])['live_count'].transform('sum')
location_monthly['num live locations on parent 12 months'] = location_monthly.groupby(['highest_parent_id', 'month'])['live_count 12 months'].transform('sum')
location_monthly['ec_saas_arr'] = location_monthly['ec_saas_arr'] / location_monthly['num live locations on parent']
location_monthly['ec_saas_arr 12 months'] = location_monthly['ec_saas_arr 12 months'] / location_monthly['num live locations on parent 12 months']

location_monthly['ec_saas_arr'].fillna(0, inplace=True)
location_monthly['ec_saas_arr 12 months'].fillna(0, inplace=True)

location_monthly = location_monthly[location_monthly['month'] > location_monthly['parent_first_go_live']]
location_monthly['first_full_month'] = location_monthly['parent_first_go_live'] + relativedelta(months=1)
location_monthly['first_full_month'] = location_monthly['first_full_month'].apply(lambda v: date(v.year, v.month, 1))
location_monthly['quarter'] = location_monthly['month'].apply(lambda v: get_first_day_of_the_quarter(v))


In [25]:
query = f"""SELECT customer_id, latest_nps_score
            FROM {database}.cs_restaurant_success.customer_nps_aggregated
            WHERE latest_nps_survey_date >= DATEADD('Months', -6, CURRENT_DATE)
        """
nps = QueryDWH(query, cnx, ['snowflake_id', 'nps_score'])


In [26]:
historic_arpu = pd.DataFrame()
month_list = location_monthly[location_monthly['month'] >= date(2016,1,1)]['month'].unique().tolist()
month_list.sort()
for month in month_list:
    tmp = location_monthly[location_monthly['month'] == month].merge(nps, on='snowflake_id', how='left')
    tmp['month'] = pd.to_datetime(tmp['month']).dt.date
    tmp = tmp[tmp['market_segment'] == 'SMB']

    tmp['Has Guest'] = np.where((tmp['has_guest_saas']) > 0, True, False)
    tmp['Has Guest 12 Months'] = np.where((tmp['has_guest_saas 12 months']) > 0, True, False)
    tmp['Has EC'] = np.where(tmp['ec_saas_arr'] > 0, True, False)
    tmp['Has EC 12 Months'] = np.where(tmp['ec_saas_arr 12 months'] > 0, True, False)
    tmp['Product Line'] = 'POS'
    tmp['Product Line'] = np.where((tmp['Has Guest']) & (tmp['Has EC'] == False), 'POS + Guest', tmp['Product Line'])
    tmp['Product Line'] = np.where((tmp['Has Guest'] == False) & (tmp['Has EC']), 'POS + EC', tmp['Product Line'])
    tmp['Product Line'] = np.where((tmp['Has Guest']) & (tmp['Has EC']), 'POS + Guest + EC', tmp['Product Line'])
    tmp['Product Line'].value_counts(normalize=True)

    tmp['Promoter Count'] = np.where(tmp['nps_score'].notnull(), 0,  np.nan)
    tmp['Promoter Count'] = np.where(tmp['nps_score'].isin([9, 10]), 1,  tmp['Promoter Count'])
    tmp['Detractor Count'] = np.where(tmp['nps_score'].notnull(), 0,  np.nan)
    tmp['Detractor Count'] = np.where(tmp['nps_score'].isin([0,1,2,3,4,5,6]), 1,  tmp['Detractor Count'])
    tmp['NPS Count'] = np.where(tmp['nps_score'].notnull(), 1,  np.nan)
    month = pd.to_datetime(month).date()
    tmp['Month'] = month
    agg = tmp.groupby(['Product Line']).agg({'snowflake_id': 'count', 
                                           'payments_margin_arr_full_month': 'sum', 
                                           'pos_live_saas_arr_full_month': 'sum', 
                                           'ec_saas_arr': 'sum', 
                                           'Promoter Count': 'sum', 
                                           'Detractor Count': 'sum', 
                                           'NPS Count': 'sum'})

    agg.rename(columns = {'snowflake_id': 'Locations', 
                          'payments_margin_arr_full_month': 'Payments ARR', 
                          'pos_live_saas_arr_full_month': 'POS SaaS', 
                          'ec_saas_arr': 'EC SaaS'}, inplace=True)

    total_row = agg.sum(numeric_only=True)
    
    total_row.rename('Total', inplace=True)
    agg = agg.append(total_row)
    agg['NPS'] = 100 * (agg['Promoter Count'] - agg['Detractor Count']) / agg['NPS Count']
    agg['# of NPS Surveys'] = agg['NPS Count'].astype(int)
    agg.drop(['Promoter Count', 'Detractor Count', 'NPS Count'], axis=1, inplace=True, errors='ignore')

    agg['Payments ARPU'] = agg['Payments ARR'] / agg['Locations']
    agg['SaaS ARPU'] = (agg['POS SaaS'] + agg['EC SaaS']) / agg['Locations']
    agg['Total ARPU'] = agg['Payments ARPU'] + agg['SaaS ARPU']
    agg['Month'] = month
    agg.set_index('Month', append=True, inplace=True)
    agg = agg.reorder_levels(['Month', 'Product Line'], axis = 'index')
    historic_arpu = pd.concat([historic_arpu, agg])
#     display(agg)

if save_bool:
    historic_arpu.to_csv(os.path.join(output_folder, 'Historic ARPU By Products.csv'), index=True, encoding='utf-8')

display(historic_arpu.tail(10))

Locations  Payments ARR      POS SaaS  \
Month      Product Line                                              
2022-05-01 POS                  5271.0  2.344153e+07  1.091001e+07   
           POS + EC              408.0  1.425137e+06  9.431892e+05   
           POS + Guest         43430.0  3.325772e+08  2.041183e+08   
           POS + Guest + EC     5946.0  4.675191e+07  3.128561e+07   
           Total               55055.0  4.041957e+08  2.472571e+08   
2022-06-01 POS                  5484.0  2.340228e+07  1.133344e+07   
           POS + EC              429.0  1.627908e+06  9.897783e+05   
           POS + Guest         44813.0  3.354223e+08  2.125319e+08   
           POS + Guest + EC     6324.0  4.865983e+07  3.362027e+07   
           Total               57050.0  4.091123e+08  2.584754e+08   

                                  EC SaaS        NPS  # of NPS Surveys  \
Month      Product Line                                                  
2022-05-01 POS               0.000000e+00  20.476190               630   
           POS + EC          9.112787e+05  26.315789                57   
           POS + Guest       0.000000e+00  21.946997              7509   
           POS + Guest + EC  1.962597e+07  18.400688              1163   
           Total             2.053724e+07  21.433914              9359   
2022-06-01 POS               0.000000e+00  19.969278               651   
           POS + EC          1.044567e+06  27.868852                61   
           POS + Guest       0.000000e+00  22.112511              7697   
           POS + Guest + EC  2.149755e+07  18.986323              1243   
           Total             2.254211e+07  21.601741              9652   

                             Payments ARPU    SaaS ARPU    Total ARPU  
Month      Product Line                                                
2022-05-01 POS                 4447.263656  2069.817264   6517.080919  
           POS + EC            3492.981836  4545.264504   8038.246339  
           POS + Guest         7657.775058  4699.938367  12357.713425  
           POS + Guest + EC    7862.748967  8562.323234  16425.072200  
           Total               7341.671773  4864.124456  12205.796229  
2022-06-01 POS                 4267.375088  2066.637249   6334.012338  
           POS + EC            3794.657147  4742.063075   8536.720222  
           POS + Guest         7484.932175  4742.639190  12227.571366  
           POS + Guest + EC    7694.470772  8715.657167  16410.127939  
           Total               7171.118171  4925.810512  12096.928682

In [30]:
# def get_investor_dataset()
cols = ['month', 'salesforce_accountid',  'highest_parent_id', 'customer_id', 'status', 'fod', 'close_date','churn_date', 'go_live',
        'booked','pos_backlog_saas_arr', 'pos_live_saas_arr', 'rev_share_arr', 'monthly_payments_margin', 'monthly_gmv', 'monthly_gpv']
investor_dataset = location_pos_monthly[(location_pos_monthly['status'].isin(['Backlog', 'Go-Live', 'Live','Churn', 'Canceled Contract']))][cols]
investor_dataset['fod_month'] = pd.to_datetime(investor_dataset['fod'].fillna(date(2000,1,1))).apply(lambda v: date(int(v.year), int(v.month), 1))

investor_dataset['status'] = np.where(investor_dataset['fod_month'] == investor_dataset['month'], 'Go-Live Month', investor_dataset['status'])

investor_dataset['backlog_saas'] = investor_dataset['pos_backlog_saas_arr'] / 12
investor_dataset['live_saas'] = investor_dataset['pos_live_saas_arr'] / 12

investor_dataset['pre_live_saas'] = np.where(investor_dataset['status'].isin(['Backlog']), investor_dataset['live_saas'], 0)
investor_dataset['live_saas'] = np.where(investor_dataset['status'].isin(['Backlog']), 0, investor_dataset['live_saas'] )


investor_dataset_wide = investor_dataset[['salesforce_accountid' ,'month', 'status', 'backlog_saas', 'pre_live_saas', 'live_saas', 'monthly_payments_margin', 'monthly_gmv']]

investor_dataset_wide['monthly_gmv'] = np.where(investor_dataset_wide['status'].isin(['Go-Live Month', 'Live']), investor_dataset_wide['monthly_gmv'], 0)
investor_dataset_wide['monthly_payments_margin'] = np.where(investor_dataset_wide['status'].isin(['Go-Live Month', 'Live']), investor_dataset_wide['monthly_payments_margin'], 0)
investor_dataset_wide = investor_dataset_wide[['salesforce_accountid', 'month', 'status', 'backlog_saas', 'pre_live_saas', 'live_saas', 'monthly_gmv', 'monthly_payments_margin']].set_index(['salesforce_accountid', 'month'])
investor_dataset_wide = investor_dataset_wide.unstack(level=-1)
investor_dataset_wide.columns = investor_dataset_wide.columns.swaplevel(0, 1)
investor_dataset_wide.sort_index(axis=1, inplace=True)

ec_monthly_wide = ec_monthly_parent_flat.groupby(['highest_parent_id', 'ec_guid','month'])[['ec_saas_arr']].sum().rename(columns = {'ec_saas_arr': 'Monthly Live EC SaaS'})
ec_monthly_wide['Monthly Live EC SaaS'] = ec_monthly_wide['Monthly Live EC SaaS'] / 12
ec_monthly_wide = ec_monthly_wide.unstack(level=-1)
ec_monthly_wide.columns = ec_monthly_wide.columns.swaplevel(0, 1)
ec_monthly_wide.sort_index(axis=1, inplace=True)

ec_acct_first_order = ec_accts[ec_accts['ec_guid'].isin(ec_monthly_parent_flat['ec_guid'].unique().tolist())]
"""
Get the first time there was backlog or live status 
"""
min_booked_month = investor_dataset[investor_dataset['status'].isin(['Backlog', 'Live'])].groupby(['salesforce_accountid'])[['month']].min().reset_index().rename(columns={'month': 'min_book_month'})
min_live_month = investor_dataset[investor_dataset['status'].isin(['Live'])].groupby(['salesforce_accountid'])[['month']].min().reset_index().rename(columns={'month': 'min_live_month'})
"""
Find the last row of data and use that as the most recent data point
"""
status_grouping = investor_dataset.sort_values('month', ascending=False).groupby(['salesforce_accountid'])

most_recent_status = status_grouping.head(1).sort_values('month', ascending=False).drop_duplicates().reset_index(drop=True)
most_recent_status = most_recent_status.merge(min_booked_month, how='left').merge(min_live_month, how='left')
most_recent_status = most_recent_status[['salesforce_accountid', 'highest_parent_id', 'status', 'fod', 'close_date', 'churn_date', 'min_book_month', 'min_live_month']]


most_recent_status['close_month'] = pd.to_datetime(most_recent_status['close_date'].fillna(date(2000,1,1))).apply(lambda v: date(v.year, v.month, 1))
most_recent_status['close_month'] = np.where(most_recent_status['close_month'] == date(2000,1,1), np.nan, most_recent_status['close_month'])
most_recent_status['fod_month'] = pd.to_datetime(most_recent_status['fod'].fillna(date(2000,1,1))).apply(lambda v: date(int(v.year), int(v.month), 1))
most_recent_status['fod_month'] = np.where(most_recent_status['fod_month'] == date(2000,1,1), np.nan, most_recent_status['fod_month'])

for col in ['close_month', 'fod_month', 'min_book_month', 'min_live_month']:
    most_recent_status[col] = pd.to_datetime(most_recent_status[col]).dt.date

most_recent_status['fod_adj'] = np.where(most_recent_status['min_live_month'] < most_recent_status['fod_month'], most_recent_status['min_live_month'], most_recent_status['fod'])

most_recent_status['close_date_adj'] = np.where(most_recent_status['close_date'] > most_recent_status['fod_adj'], most_recent_status['fod_adj'].fillna(date(2013,1,2)) - relativedelta(days=1), most_recent_status['close_date'])
most_recent_status['close_date_adj'].fillna(date(2099,1,1), inplace=True)
most_recent_status['close_date_adj'] = np.where(most_recent_status['min_book_month'] < most_recent_status['close_month'], most_recent_status['min_book_month'], most_recent_status['close_date_adj'])
most_recent_status.rename(columns = {'status': 'Most Recent Status', 'close_date_adj': 'Booking Date', 'fod_adj': 'Go-Live'}, inplace=True)
most_recent_status = most_recent_status[['salesforce_accountid', 'highest_parent_id', 'Most Recent Status', 'Booking Date', 'Go-Live']]

most_recent_status = most_recent_status.merge(acct_info[['sf_accountid', 'customer_name', 'market_segment', 'restaurant_type', 'state', 'parent_first_go_live']].rename(columns = {'sf_accountid': 'salesforce_accountid', 'parent_first_go_live': 'First Go-Live on Parent'}), how='left')
most_recent_status = most_recent_status.merge(acct_info[['customer_id', 'customer_name']].rename(columns = {'customer_id': 'highest_parent_id', 'customer_name': 'Parent Name'}), how='left')
most_recent_status.rename(columns = {'customer_name': 'Location Name', 'market_segment': 'Market Segment', 'restaurant_type': 'Restaurant Type', 'state': 'State'}, inplace=True)

if save_bool:
    writer = pd.ExcelWriter(os.path.join(output_folder, 'Investor Dataset.xlsx'), engine='xlsxwriter')
    most_recent_status.to_excel(writer, sheet_name='Account Info', index=True)
    investor_dataset_wide.to_excel(writer, sheet_name='Monthly Location Data', index=True)
    ec_monthly_wide.to_excel(writer, sheet_name='Monthly EC Data', index=True)
    ec_acct_first_order.to_excel(writer, sheet_name='EC Account Info', index=True)
    writer.save()    
    investor_dataset.to_csv(os.path.join(output_folder, 'Investor Dataset Long.csv'), index=False)
    ec_monthly_parent_flat.to_csv(os.path.join(output_folder, 'Employee Cloud Long.csv'), index=False)

In [27]:
# QA Check 
qa_2020 = parent_monthly_keeping_churn[pd.to_datetime(parent_monthly_keeping_churn['month']).dt.year == 2020]
qa_2020.groupby('month')[['live_count', 'pos_live_saas_arr_full_month', 'ec_saas_arr', 'payments_margin_arr_full_month']].sum()

,live_count,pos_live_saas_arr_full_month,ec_saas_arr,payments_margin_arr_full_month
month,,,,
2020-01-01,26572,7.854573e+07,4748739.84,9.623425e+07
2020-02-01,27950,8.284049e+07,4691749.56,1.109771e+08
2020-03-01,29169,8.827374e+07,5142477.36,8.314340e+07
2020-04-01,30211,9.208423e+07,0.00,6.879067e+07
2020-05-01,30507,9.415554e+07,4130033.16,1.091892e+08
2020-06-01,31319,9.775470e+07,4763558.88,1.401864e+08
2020-07-01,32736,1.032442e+08,5561710.08,1.601991e+08
2020-08-01,34090,1.079094e+08,5760048.84,1.796899e+08
2020-09-01,35288,1.121666e+08,6165724.80,1.747964e+08


# Net Retention Rate Aggregated
## Get NRR on an aggregate and Market Segment Level

NRR is Net Revenue Retention - It measures how much revenue is retained from a base cohort <br />
For example: If we want to see the NRR from Nov 2020 to Nov 2021 (always Year over Year) <br />
We will look at all customers live in Nov 2020 and get their revenue in that month, and for the same <br />
customers, we will see their revenue in Nov 2021 <br /> <br />

NRR includes Payments, SaaS (POS and EC) and accounts for upsells, downsells, churn and account expansion <br />
however, new customers that are added in between the two months are not included. but accounts expansions for the <br />
customers that are already on Toast do get included


In [30]:
save_raw = False
nrr_agg_removing_churn = retention(parent_monthly_removing_churn, save_bool, output_folder, 'NRR Removing Churn.xlsx', retention_type = 'Net', save_raw = save_raw)
grr_agg_removing_churn, rr = retention(parent_monthly_removing_churn, save_bool, output_folder, 'GRR Removing Churn.xlsx', retention_type = 'Gross', save_raw = save_raw)


grr_agg_keeping_churn, rr = retention(parent_monthly_keeping_churn, save_bool, output_folder, 'GRR Keeping Churn.xlsx', retention_type = 'Gross', save_raw = save_raw)
nrr_agg_keeping_churn, nrr_raw = retention(parent_monthly_keeping_churn, save_bool, output_folder, 'NRR Keeping Churn.xlsx', retention_type = 'Net', save_raw = save_raw)

In [32]:
nrr_agg_keeping_churn['Total Start'] = nrr_agg_keeping_churn['Starting Live SaaS'] + nrr_agg_keeping_churn['Payments ARR']
nrr_agg_keeping_churn['Total End'] = nrr_agg_keeping_churn['Ending Live SaaS'] + nrr_agg_keeping_churn['Payments ARR 12 Months']
for year in [2014, 2015, 2016, 2017, 2018, 2019, 2020]:
    print(year)
    tmp = nrr_agg_keeping_churn[pd.to_datetime(nrr_agg_keeping_churn.index.to_series()).dt.year == year]
    tmp = tmp[pd.to_datetime(nrr_agg_keeping_churn.index.to_series()).dt.date < date(2021,1,1)]
    print(tmp['Total End'].sum() / tmp['Total Start'].sum())
    print('\n')

2014
1.3254459156132308


2015
1.1797640577311777


2016
1.2216031712600988


2017
1.1397499804841955


2018
1.1059245550622012


2019
1.146200718362705


2020
1.3506693366251963


